## Build ReAct Agent with Tool Calling and Memory

In [82]:
import datetime
import inspect
import json
from typing import List, Dict, Literal, Callable, Any, TypedDict
from typing import get_type_hints

from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
)

In [83]:
load_dotenv()
client = OpenAI()

### Memory Layer

We use the previous created `Memory` class in this [02-function-calling.ipynb](./02-function-calling.ipynb) to store the conversation history and the tool calls with some modifications

In [84]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(
        self,
        role: Literal[
            "user", "system", "assistant", "tool"
        ],  # Added 'tool' as a new role type
        content: str,
        tool_calls: List = None,  # New parameter to store tool call information
        tool_call_id=None,
    ) -> None:  # New parameter to track specific tool call IDs

        # For regular messages (user/system/assistant), include tool_calls dictionary
        message = {"role": role, "content": content}

        # Only add tool_calls if they exist and role is assistant
        if tool_calls and role == "assistant":
            message["tool_calls"] = tool_calls

        # Add tool_call_id if it exists and role is tool
        if tool_call_id and role == "tool":
            message["tool_call_id"] = tool_call_id

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

### Tool Layer

In previous notebooks, we explicitly defined the tool as a list and pass it into OpenAI API. However, in this notebook, we will use a class to define the tool. This class will be used to generate the tool specification and call the tool function.

Before we implement it, let's define some helper functions to infer the information used in the tool specification. The tool specification is a JSON object that contains the following information:
- 🔒`type`: The type of the tool. In this case, it is always `function`.
- `function`: The function specification. It contains the following information:
    - 🔍`name`: The name of the function.
    - 🔍`description`: The description of the function.
    - 🔒`parallel_tool_calls`: Whether the function can be called in parallel. In this case, it is always `False`.
    - `parameters`: The parameters of the function. It contains the following information:
        - 🔒`type`: The type of the parameter. In this case, it is always `object`.
        - `properties`: The properties of the parameter. It contains the following information:
            - 🔍`name`: The name of the parameter.
                - 🔍`type`: The type of the parameter. In this case, it is always `string`.
        - 🔍`required`: The required parameters of the function. In this case, it is always `[]`.
        - 🔒`additionalProperties`: Whether the function can have additional properties. In this case, it is always `False`.
    - 🔒`strict`: True

So, except for fields (🔒), we need to infer the rest of the fields (🔍)  from the function definition. Let's create some helper function to infer these information.

In [85]:
# Create simple test function
def add(a: int, b: int = 1) -> int:
    """This is a simple function that adds two numbers"""
    return a + b

**Function Name Inference**

- We can use the `__name__` attribute of the function to get the function name. This is a simple way to get the function name.

In [86]:
def get_func_name(func: Callable) -> str:
    return func.__name__


get_func_name(add)

'add'

**Function Description Inference**

- We can use the `__doc__` attribute of the function to get the function description. This is a simple way to get the function description. If the function does not have a docstring, we will use the function name as the description.

In [87]:
def get_func_desc(func: Callable) -> str:
    if func.__doc__:
        return func.__doc__.strip()
    else:
        return get_func_name(func)


get_func_desc(add)

'This is a simple function that adds two numbers'

**Arguments Type Convert**

- In tool specification, we accept the type of the parameter as `string`, `number`, `integer`, `boolean`, `object`, `array`, `null`. We will use the following mapping to map the type hints to the tool specification type.

In [88]:
def infer_json_type(arg_type: Any) -> str:
    """
    Infers the JSON schema type from a given Python type.

    Parameters:
    - arg_type: The Python type to be mapped to a JSON schema type.

    Returns:
    - str: The corresponding JSON schema type as a string.
    """
    type_mapping = {
        bool: "boolean",
        int: "integer",
        float: "number",
        str: "string",
        list: "array",
        dict: "object",
        type(None): "null",
        datetime.date: "string",
        datetime.datetime: "string",
    }

    # Check if arg_type is directly in the mapping
    if arg_type in type_mapping:
        return type_mapping[arg_type]

    # If arg_type is a subclass of a mapped type, return the mapped type
    for base_type in type_mapping:
        if isinstance(arg_type, base_type):
            return type_mapping[base_type]

    # Default to string if type is unknown
    return "string"

In [89]:
print(infer_json_type(int))
print(infer_json_type(datetime.datetime))
print(infer_json_type(None))

integer
string
null


**Arguments Name and Type Inference**

- Firstly, we will leverage the typing module to get the type hints of the function. Then we will use the type hints to generate the tool specification. Here is a simple example.

**Argument Required Inference**

- We can inspect the parameters default value exist or not to automatically decide whether the argument is required or not. If the default value is not provided, we will set the argument as required. Otherwise, we will set the argument as optional.

In [90]:
signature = inspect.signature(add)
print(signature)
print(
    "Is parameter 'a' required?",
    signature.parameters["a"].default == inspect.Parameter.empty,
)
print(
    "Is parameter 'b' required?",
    signature.parameters["b"].default == inspect.Parameter.empty,
)

(a: int, b: int = 1) -> int
Is parameter 'a' required? True
Is parameter 'b' required? False


In [91]:
class FuncArgument(TypedDict):
    name: str
    type: str
    required: bool

In [92]:
def get_func_args(func: Callable) -> List[FuncArgument]:
    args_type_mapping = get_type_hints(func)
    signature = inspect.signature(func)

    def is_required(param: inspect.Parameter) -> bool:
        return param.default == inspect.Parameter.empty

    arguments = []
    for arg_name, arg_type in args_type_mapping.items():
        param = signature.parameters.get(arg_name)
        if param:
            arguments.append(
                {
                    "name": arg_name,
                    "type": infer_json_type(arg_type),
                    "required": is_required(param),
                }
            )

    return arguments


get_func_args(add)

[{'name': 'a', 'type': 'integer', 'required': True},
 {'name': 'b', 'type': 'integer', 'required': False}]

Now, let's put all these together to create a tool class.

In [93]:
class Tool:
    def __init__(self, func: Callable):
        self.func = func
        self.name: str = get_func_name(func)
        self.description: str = get_func_desc(func)
        self.arguments: List[FuncArgument] = get_func_args(func)

    def to_dict(self):
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"]
                        for argument in self.arguments
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

We can test the tool class with the test function.

In [94]:
tool = Tool(add)
tool.to_dict()

{'type': 'function',
 'function': {'name': 'add',
  'description': 'This is a simple function that adds two numbers',
  'parallel_tool_calls': False,
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
   'required': ['a'],
   'additionalProperties': False},
  'strict': True}}

In [95]:
tool(a=1, b=1)

2

### ReAct Prompt

In previous notebook [03-react-prompt-technique.ipynb](./03-react-prompt-technique.ipynb), we define a specific ReAct prompt. Here we will use a template to generate the ReAct prompt by combining the tool calling. We will use the following template:

```python
react_prompt = (
    "You're an AI Agent, your role is {ROLE}, " 
    "and you need to {INSTRUCTIONS} "
    "You can answer multistep questions by sequentially calling functions. "
    "You follow a pattern of of Thought and Action. "
    "Create a plan of execution: "
    "- Use Thought to describe your thoughts about the question you have been asked. "
    "- Use Action to specify one of the tools available to you. "
    "When you think it's over call the termination tool. "
    "Never try to respond directly if the question needs a tool."
    "The actions you have are the Tools: "
    "{TOOLS}"
)
```

### ReAct Loop: Reasoning and Action

The core mechanism operates as a cycle of thought and action: First, the AI engages in a reasoning step without any tools to analyze the current situation. Then, it enters an iterative loop where it can both think and use tools to accomplish tasks. In each iteration, the AI:
1. 🤔 Thinks about the current state
2. 💭 Formulates a response
3. 🛠️ Decides whether to use tools
4. 📝 Records its thoughts and actions
5. 🔄 Repeats until reaching a solution or max iterations

This creates a natural flow of "Think → Act → Observe → Think" that mimics human problem-solving patterns, allowing the AI to break down complex tasks into manageable steps while maintaining a record of its reasoning process.

We can add two extra method to the Agent class (at the top of previous notebook [02-function-calling.ipynb](./02-function-calling.ipynb) ) to handle the ReAct loop and create a Exception to terminate the loop.

In [133]:
TERMINATION_MESSAGE = "StopReactLoopException"

class StopReactLoopException(Exception):
    """
    Terminates ReAct loop
    """

def termination() -> str:
    """Terminate the ReAct loop. If the agent thinks there's no further actions to take"""
    return TERMINATION_MESSAGE

In [125]:
class Agent:
    """A tool-calling AI Agent"""

    def __init__(
        self,
        name: str = "Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        funcs: List[Callable] = [],
    ):
        # Agent basic info
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.client = OpenAI()

        # Tools
        tools = [Tool(func) for func in funcs] + [Tool(termination)]
        self.tool_map = {tool.name: tool for tool in tools}
        self.tools = [tool.to_dict() for tool in tools] if funcs else None
        self.termination_message = TERMINATION_MESSAGE

        # Memory
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, "
            f"and you need to {self.instructions} "
            "You can answer multistep questions by sequentially calling functions. "
            "You follow a pattern of of Thought and Action. "
            "Create a plan of execution: "
            "- Use Thought to describe your thoughts about the question you have been asked. "
            "- Use Action to specify one of the tools available to you. "
            "When you think it's over call the termination tool. "
            "Never try to respond directly if the question needs a tool."
            "The actions you have are the Tools: "
            f"{self.tools}",
        )

    def invoke(self, user_message: str, max_iter: int = 3) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )
        try:
            self._react_loop(max_iter)
        except StopReactLoopException as e:
            print(f"Termninated loop with message: '{e!s}'")
            self._reason()

        return self.memory.last_message()

    def _react_loop(self, max_iter: int):
        for i in range(max_iter):
            self._reason()

            ai_message = self._get_completion(
                messages=self.memory.get_messages(),
                tools=self.tools,
            )
            tool_calls = ai_message.tool_calls

            self.memory.add_message(
                role="assistant",
                content=ai_message.content,
                tool_calls=tool_calls,
            )

            if tool_calls:
                self._call_tools(tool_calls)

    def _reason(self):
        # No tools
        ai_message = self._get_completion(
            messages=self.memory.get_messages(),
        )
        tool_calls = ai_message.tool_calls

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

    def _call_tools(self, tool_calls: List[ChatCompletionMessageToolCall]):
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool", content=str(result), tool_call_id=tool_call.id
            )
            if result == TERMINATION_MESSAGE:
                raise StopReactLoopException

    def _get_completion(
        self, messages: List[Dict], tools: List = None
    ) -> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=tools,
        )

        return response.choices[0].message

In [126]:
def power(base: float, exponent: float):
    """Exponentatiation: base to the power of exponent"""

    return base**exponent

In [127]:
def sum(number_1: float, number_2: float):
    """Sum / Addition: Add two numbers"""

    return number_1 + number_2

In [129]:
agent = Agent(funcs=[power, sum])

In [130]:
agent.invoke("What's 2 to the power of 3? Then add 10 to the result")

Termninated loop with message: ''


{'role': 'assistant',
 'content': 'The final result of \\(2\\) to the power of \\(3\\) plus \\(10\\) is \\(18\\). If you have any more questions or need further assistance, feel free to ask!'}

In [131]:
agent.memory.get_messages()

[{'role': 'system',
  'content': 'You\'re an AI Agent, your role is Personal Assistant, and you need to Help users with any question You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. When you think it\'s over call the termination tool. Never try to respond directly if the question needs a tool.The actions you have are the Tools: [{\'type\': \'function\', \'function\': {\'name\': \'power\', \'description\': \'Exponentatiation: base to the power of exponent\', \'parallel_tool_calls\': False, \'parameters\': {\'type\': \'object\', \'properties\': {\'base\': {\'type\': \'number\'}, \'exponent\': {\'type\': \'number\'}}, \'required\': [\'base\', \'exponent\'], \'additionalProperties\': False}, \'strict\': True}}, {\'type\': \'function\', \'function\': {\'name\'

- As you can see above, the memory shows the thought process of the agent. The agent first thought about the question, then it called the power tool to calculate the power of 2 to the power of 3. Then it called the sum tool to add 10 to the result. The agent then terminated the loop by calling the termination tool.

In [132]:
agent.invoke("What's 4 to the power of 3? Then add 10 to the result")

Termninated loop with message: ''


{'role': 'assistant',
 'content': 'The final result of \\(4\\) to the power of \\(3\\) plus \\(10\\) is \\(74\\). If you have any more questions or need further assistance, feel free to ask!'}